In [1]:
# In this workshop we perform topic modeling using gensim

# We are using the subnews dataset that we used last week. 
# The "Class" labels here are only used for sanity check of the topics discovered later.
# Remember, in actual use of topic modelling, the documents DON'T come with labeled classes.
# It's unsupervised learning.

import pandas as pd
news=pd.read_table('r8-train-all-terms.txt',header=None,names = ["Class", "Text"])
subnews=news[(news.Class=="trade")| (news.Class=='crude')|(news.Class=='money-fx') ]
subnews.head()

,Class,Text
15,trade,brazil anti inflation plan limps to anniversar...
43,crude,diamond shamrock dia cuts crude prices diamond...
55,crude,opec may have to meet to firm prices analysts ...
76,crude,texaco canada cuts crude prices canadian cts b...
77,crude,texaco canada txc lowers crude postings texaco...


In [2]:
# Let's use the similar preprocessing we used last week.
# The output of each document is a list of tokens.

import nltk
from nltk.corpus import stopwords
mystopwords=stopwords.words("English") + ['one', 'become', 'get', 'make', 'take']
WNlemma = nltk.WordNetLemmatizer()

def pre_process(text):
    tokens = nltk.word_tokenize(text)
    tokens=[ WNlemma.lemmatize(t.lower()) for t in tokens]
    tokens=[ t for t in tokens if t not in mystopwords]
    tokens = [ t for t in tokens if len(t) >= 3 ]
    return(tokens)

In [3]:
#split the data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(subnews.Text, subnews.Class, test_size=0.33, random_state=12)


In [4]:
# Apply preprocessing to every document in the training set.
toks_train = X_train.apply(pre_process)


In [5]:
#!pip install gensim

In [6]:
import logging
import gensim 
from gensim import corpora

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Users\isscy\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
# Prepare a vocabulary dictionary.
dictionary = corpora.Dictionary(toks_train)
print(dictionary)

2018-09-15 08:34:20,736 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-15 08:34:20,844 : INFO : built Dictionary(5971 unique tokens: ['aide', 'amendment', 'annual', 'approved', 'approves']...) from 475 documents (total 57987 corpus positions)


Dictionary(5971 unique tokens: ['aide', 'amendment', 'annual', 'approved', 'approves']...)


In [8]:
# Filter off any words with document frequency less than 3, or appearing in more than 80% documents
dictionary.filter_extremes(no_below=3, no_above=0.7)
print(dictionary)

2018-09-15 08:34:20,859 : INFO : discarding 3537 tokens: [('frightening', 2), ('mandating', 2), ('reuter', 408), ('said', 403), ('toughening', 2), ('wedge', 2), ('aggravated', 1), ('delay', 1), ('devaluation', 2), ('herald', 1)]...
2018-09-15 08:34:20,862 : INFO : keeping 2434 tokens which were in no less than 3 and no more than 332 (=70.0%) documents
2018-09-15 08:34:20,866 : INFO : resulting dictionary: Dictionary(2434 unique tokens: ['aide', 'amendment', 'annual', 'approved', 'approves']...)


Dictionary(2434 unique tokens: ['aide', 'amendment', 'annual', 'approved', 'approves']...)


In [9]:
type(dictionary)

gensim.corpora.dictionary.Dictionary

In [10]:
# Use the dictionary to prepare a DTM (using TF)
dtm_train = [dictionary.doc2bow(d) for d in toks_train ]

In [11]:
# Build an LDA model for 3 topics out of the DTM
%time lda = gensim.models.ldamodel.LdaModel(dtm_train, num_topics = 3,  alpha='auto',chunksize=30,id2word = dictionary, passes = 20,random_state=432)
#chunksize (int, optional) – Number of documents to be used in each training chunk.
#passes (int, optional) – Number of passes through the corpus during training.

2018-09-15 08:34:21,054 : INFO : using autotuned alpha, starting with [0.33333334, 0.33333334, 0.33333334]
2018-09-15 08:34:21,056 : INFO : using symmetric eta at 0.3333333333333333
2018-09-15 08:34:21,058 : INFO : using serial LDA version on this node
2018-09-15 08:34:21,062 : INFO : running online (multi-pass) LDA training, 3 topics, 20 passes over the supplied corpus of 475 documents, updating model once every 30 documents, evaluating perplexity every 300 documents, iterating 50x with a convergence threshold of 0.001000
2018-09-15 08:34:21,062 : INFO : PROGRESS: pass 0, at document #30/475
2018-09-15 08:34:21,111 : INFO : optimized alpha [0.5270892, 0.50643796, 0.48903793]
2018-09-15 08:34:21,114 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 08:34:21,121 : INFO : topic #0 (0.527): 0.027*"trade" + 0.013*"china" + 0.013*"oil" + 0.012*"year" + 0.011*"export" + 0.011*"would" + 0.009*"last" + 0.009*"gatt" + 0.008*"system" + 0.008*"currency"
2018-09-1

2018-09-15 08:34:21,586 : INFO : optimized alpha [0.5715378, 0.47655696, 0.52661896]
2018-09-15 08:34:21,587 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 08:34:21,590 : INFO : topic #0 (0.572): 0.028*"trade" + 0.017*"japan" + 0.015*"would" + 0.009*"year" + 0.008*"world" + 0.008*"agreement" + 0.007*"minister" + 0.007*"import" + 0.007*"foreign" + 0.007*"market"
2018-09-15 08:34:21,591 : INFO : topic #1 (0.477): 0.046*"oil" + 0.029*"mln" + 0.018*"price" + 0.014*"barrel" + 0.014*"opec" + 0.014*"pct" + 0.013*"market" + 0.012*"production" + 0.010*"stg" + 0.010*"bank"
2018-09-15 08:34:21,592 : INFO : topic #2 (0.527): 0.020*"billion" + 0.015*"dlrs" + 0.015*"dollar" + 0.015*"bank" + 0.014*"japan" + 0.014*"official" + 0.014*"japanese" + 0.011*"market" + 0.009*"trade" + 0.009*"yen"
2018-09-15 08:34:21,593 : INFO : topic diff=0.477447, rho=0.333333
2018-09-15 08:34:21,639 : INFO : -7.282 per-word bound, 155.7 perplexity estimate based on a held-out corpus of

2018-09-15 08:34:22,027 : INFO : topic #1 (0.398): 0.043*"oil" + 0.025*"mln" + 0.022*"price" + 0.018*"opec" + 0.017*"bpd" + 0.016*"crude" + 0.012*"dlrs" + 0.012*"barrel" + 0.011*"pct" + 0.009*"market"
2018-09-15 08:34:22,028 : INFO : topic #2 (0.366): 0.028*"billion" + 0.018*"dlrs" + 0.018*"bank" + 0.015*"official" + 0.013*"japanese" + 0.011*"dollar" + 0.010*"market" + 0.010*"february" + 0.009*"year" + 0.008*"last"
2018-09-15 08:34:22,028 : INFO : topic diff=0.275793, rho=0.236801
2018-09-15 08:34:22,029 : INFO : PROGRESS: pass 1, at document #60/475
2018-09-15 08:34:22,057 : INFO : optimized alpha [0.42603716, 0.3405189, 0.33244377]
2018-09-15 08:34:22,057 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 08:34:22,060 : INFO : topic #0 (0.426): 0.033*"trade" + 0.013*"would" + 0.012*"country" + 0.011*"japan" + 0.008*"year" + 0.007*"state" + 0.007*"foreign" + 0.007*"agreement" + 0.006*"economic" + 0.006*"market"
2018-09-15 08:34:22,061 : INFO : topic #1

2018-09-15 08:34:22,400 : INFO : PROGRESS: pass 1, at document #300/475
2018-09-15 08:34:22,422 : INFO : optimized alpha [0.33410797, 0.27763724, 0.29562563]
2018-09-15 08:34:22,423 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 08:34:22,426 : INFO : topic #0 (0.334): 0.031*"trade" + 0.016*"japan" + 0.016*"would" + 0.009*"world" + 0.008*"year" + 0.008*"agreement" + 0.008*"country" + 0.007*"minister" + 0.007*"market" + 0.006*"foreign"
2018-09-15 08:34:22,427 : INFO : topic #1 (0.278): 0.048*"oil" + 0.026*"mln" + 0.020*"price" + 0.014*"opec" + 0.014*"dlrs" + 0.013*"barrel" + 0.013*"pct" + 0.011*"crude" + 0.011*"market" + 0.010*"stg"
2018-09-15 08:34:22,428 : INFO : topic #2 (0.296): 0.024*"billion" + 0.020*"bank" + 0.017*"dollar" + 0.016*"dlrs" + 0.014*"japanese" + 0.013*"official" + 0.012*"market" + 0.011*"japan" + 0.010*"trade" + 0.009*"chip"
2018-09-15 08:34:22,429 : INFO : topic diff=0.243499, rho=0.236801
2018-09-15 08:34:22,429 : INFO : PROGRESS

2018-09-15 08:34:22,790 : INFO : topic #1 (0.230): 0.043*"oil" + 0.025*"mln" + 0.022*"price" + 0.019*"opec" + 0.017*"bpd" + 0.016*"crude" + 0.014*"dlrs" + 0.013*"barrel" + 0.009*"ecuador" + 0.009*"saudi"
2018-09-15 08:34:22,791 : INFO : topic #2 (0.226): 0.030*"billion" + 0.024*"bank" + 0.019*"dollar" + 0.016*"dlrs" + 0.014*"official" + 0.012*"japanese" + 0.010*"february" + 0.010*"market" + 0.010*"january" + 0.009*"trade"
2018-09-15 08:34:22,792 : INFO : topic diff=0.258099, rho=0.230429
2018-09-15 08:34:22,793 : INFO : PROGRESS: pass 2, at document #90/475
2018-09-15 08:34:22,820 : INFO : optimized alpha [0.2803443, 0.2376004, 0.21250975]
2018-09-15 08:34:22,821 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 08:34:22,824 : INFO : topic #0 (0.280): 0.033*"trade" + 0.014*"would" + 0.011*"country" + 0.011*"japan" + 0.009*"year" + 0.007*"state" + 0.007*"agreement" + 0.007*"foreign" + 0.006*"minister" + 0.006*"new"
2018-09-15 08:34:22,825 : INFO : topic

...

2018-09-15 08:34:33,016 : INFO : topic #1 (0.128): 0.049*"oil" + 0.026*"price" + 0.018*"dlrs" + 0.017*"crude" + 0.017*"opec" + 0.016*"mln" + 0.015*"bpd" + 0.014*"barrel" + 0.011*"company" + 0.010*"pct"
2018-09-15 08:34:33,017 : INFO : topic #2 (0.158): 0.032*"bank" + 0.026*"billion" + 0.018*"mln" + 0.017*"market" + 0.015*"dollar" + 0.014*"rate" + 0.013*"currency" + 0.013*"exchange" + 0.011*"pct" + 0.010*"stg"
2018-09-15 08:34:33,017 : INFO : topic diff=0.174310, rho=0.171920
2018-09-15 08:34:33,018 : INFO : PROGRESS: pass 18, at document #30/475
2018-09-15 08:34:33,039 : INFO : optimized alpha [0.20125322, 0.13114835, 0.15523773]
2018-09-15 08:34:33,044 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 08:34:33,048 : INFO : topic #0 (0.201): 0.034*"trade" + 0.013*"japan" + 0.013*"would" + 0.011*"country" + 0.009*"japanese" + 0.008*"year" + 0.008*"state" + 0.007*"agreement" + 0.007*"official" + 0.007*"market"
2018-09-15 08:34:33,049 : INFO : topic #1 (0

2018-09-15 08:34:33,331 : INFO : optimized alpha [0.2010857, 0.13029647, 0.15958515]
2018-09-15 08:34:33,332 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 08:34:33,335 : INFO : topic #0 (0.201): 0.029*"trade" + 0.019*"japan" + 0.014*"would" + 0.011*"japanese" + 0.009*"official" + 0.009*"country" + 0.009*"year" + 0.008*"market" + 0.007*"minister" + 0.007*"agreement"
2018-09-15 08:34:33,336 : INFO : topic #1 (0.130): 0.050*"oil" + 0.025*"price" + 0.018*"mln" + 0.018*"opec" + 0.017*"dlrs" + 0.015*"barrel" + 0.014*"crude" + 0.011*"pct" + 0.011*"saudi" + 0.010*"bpd"
2018-09-15 08:34:33,337 : INFO : topic #2 (0.160): 0.031*"bank" + 0.026*"billion" + 0.021*"dollar" + 0.019*"mln" + 0.016*"market" + 0.014*"currency" + 0.013*"rate" + 0.012*"exchange" + 0.012*"pct" + 0.011*"stg"
2018-09-15 08:34:33,338 : INFO : topic diff=0.181038, rho=0.169435
2018-09-15 08:34:33,376 : INFO : -7.044 per-word bound, 131.9 perplexity estimate based on a held-out corpus of 30 d

2018-09-15 08:34:33,677 : INFO : topic #1 (0.130): 0.046*"oil" + 0.025*"price" + 0.018*"opec" + 0.017*"dlrs" + 0.017*"crude" + 0.017*"bpd" + 0.017*"mln" + 0.013*"barrel" + 0.009*"company" + 0.009*"pct"
2018-09-15 08:34:33,681 : INFO : topic #2 (0.153): 0.031*"bank" + 0.027*"billion" + 0.020*"mln" + 0.017*"market" + 0.015*"dollar" + 0.014*"currency" + 0.013*"rate" + 0.013*"exchange" + 0.012*"stg" + 0.011*"pct"
2018-09-15 08:34:33,685 : INFO : topic diff=0.155427, rho=0.167054
2018-09-15 08:34:33,687 : INFO : PROGRESS: pass 19, at document #60/475
2018-09-15 08:34:33,702 : INFO : optimized alpha [0.19882703, 0.12604125, 0.16004272]
2018-09-15 08:34:33,703 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 08:34:33,705 : INFO : topic #0 (0.199): 0.032*"trade" + 0.015*"japan" + 0.013*"would" + 0.012*"country" + 0.010*"japanese" + 0.008*"year" + 0.008*"state" + 0.008*"official" + 0.007*"market" + 0.007*"agreement"
2018-09-15 08:34:33,706 : INFO : topic #1 (0

2018-09-15 08:34:33,993 : INFO : PROGRESS: pass 19, at document #300/475
2018-09-15 08:34:34,012 : INFO : optimized alpha [0.19917622, 0.13374747, 0.1613215]
2018-09-15 08:34:34,015 : INFO : merging changes from 30 documents into a model of 475 documents
2018-09-15 08:34:34,018 : INFO : topic #0 (0.199): 0.031*"trade" + 0.019*"japan" + 0.014*"would" + 0.011*"japanese" + 0.009*"official" + 0.008*"year" + 0.008*"country" + 0.008*"market" + 0.008*"world" + 0.007*"agreement"
2018-09-15 08:34:34,018 : INFO : topic #1 (0.134): 0.050*"oil" + 0.025*"price" + 0.017*"dlrs" + 0.017*"mln" + 0.016*"opec" + 0.014*"barrel" + 0.013*"crude" + 0.011*"saudi" + 0.011*"pct" + 0.010*"bpd"
2018-09-15 08:34:34,019 : INFO : topic #2 (0.161): 0.032*"bank" + 0.026*"billion" + 0.020*"dollar" + 0.020*"mln" + 0.016*"market" + 0.014*"currency" + 0.013*"rate" + 0.012*"exchange" + 0.012*"stg" + 0.012*"dlrs"
2018-09-15 08:34:34,020 : INFO : topic diff=0.148212, rho=0.167054
2018-09-15 08:34:34,021 : INFO : PROGRESS: pa

Wall time: 13.2 s


In [12]:
# To see the topics, with the most probable words in each topic. What topics to you see? 
lda.show_topics()

[(0,
  '0.033*"trade" + 0.014*"japan" + 0.014*"would" + 0.010*"country" + 0.010*"japanese" + 0.008*"year" + 0.008*"state" + 0.007*"official" + 0.007*"agreement" + 0.007*"market"'),
 (1,
  '0.049*"oil" + 0.026*"price" + 0.018*"dlrs" + 0.017*"crude" + 0.017*"opec" + 0.016*"mln" + 0.015*"bpd" + 0.014*"barrel" + 0.011*"company" + 0.010*"pct"'),
 (2,
  '0.032*"bank" + 0.026*"billion" + 0.019*"mln" + 0.017*"market" + 0.015*"dollar" + 0.014*"rate" + 0.013*"currency" + 0.013*"exchange" + 0.011*"pct" + 0.010*"stg"')]

In [13]:
# You can also request to see more words per topic
lda.show_topics(num_words=20)

[(0,
  '0.033*"trade" + 0.014*"japan" + 0.014*"would" + 0.010*"country" + 0.010*"japanese" + 0.008*"year" + 0.008*"state" + 0.007*"official" + 0.007*"agreement" + 0.007*"market" + 0.007*"world" + 0.006*"foreign" + 0.006*"united" + 0.006*"minister" + 0.006*"new" + 0.006*"last" + 0.005*"told" + 0.005*"import" + 0.005*"industry" + 0.005*"bill"'),
 (1,
  '0.049*"oil" + 0.026*"price" + 0.018*"dlrs" + 0.017*"crude" + 0.017*"opec" + 0.016*"mln" + 0.015*"bpd" + 0.014*"barrel" + 0.011*"company" + 0.010*"pct" + 0.008*"production" + 0.008*"last" + 0.007*"output" + 0.007*"petroleum" + 0.007*"ecuador" + 0.006*"day" + 0.006*"saudi" + 0.006*"would" + 0.006*"year" + 0.006*"per"'),
 (2,
  '0.032*"bank" + 0.026*"billion" + 0.019*"mln" + 0.017*"market" + 0.015*"dollar" + 0.014*"rate" + 0.013*"currency" + 0.013*"exchange" + 0.011*"pct" + 0.010*"stg" + 0.010*"dlrs" + 0.008*"money" + 0.008*"central" + 0.008*"deficit" + 0.008*"paris" + 0.007*"system" + 0.007*"monetary" + 0.007*"year" + 0.006*"treasury" + 0.0

In [14]:
# Get the topic distribution of documents
dtopics_train = lda.get_document_topics(dtm_train)

In [15]:
# Get the topic likelihood for the first document in train set
for i in range(0, 5):
    print(dtopics_train[i])

[(0, 0.99748653)]
[(0, 0.09187004), (2, 0.9053942)]
[(0, 0.7505174), (1, 0.11056269), (2, 0.13891992)]
[(0, 0.96214074), (1, 0.016980482), (2, 0.020878794)]
[(1, 0.9644994), (2, 0.03264432)]


In [16]:
# Pick the topic with the highest probability for each document, map it to the label
# NOTE: the mapping may change in a different run
from operator import itemgetter
top_train = [ max(t, key=itemgetter(1))[0] for t in dtopics_train ]
dict = {1: 'crude', 2: 'money-fx', 0: 'trade'}
topic_train = [ dict[t] for t in top_train ]

In [17]:
# Now let's see how well these topics match the actual categories
import numpy as np
from sklearn import metrics
print(metrics.confusion_matrix(topic_train, y_train))
print(np.mean(topic_train == y_train) )
print(metrics.classification_report(topic_train, y_train))

[[153   1   1]
 [  3 107  33]
 [ 18  18 141]]
0.8442105263157895
             precision    recall  f1-score   support

      crude       0.88      0.99      0.93       155
   money-fx       0.85      0.75      0.80       143
      trade       0.81      0.80      0.80       177

avg / total       0.84      0.84      0.84       475



In [18]:
# The typical practice is to use the reserved test set for evaluation
toks_test = X_test.apply(pre_process)
dtm_test = [dictionary.doc2bow(d) for d in toks_test ]
dtopics_test = lda.get_document_topics(dtm_test)
top_test = [ max(t,key=itemgetter(1))[0] for t in dtopics_test ]
topic_test = [ dict[t] for t in top_test ]
print(metrics.confusion_matrix(topic_test, y_test))
print(np.mean(topic_test == y_test) )
print(metrics.classification_report(topic_test, y_test))

[[62  1  0]
 [ 4 66 16]
 [13 13 60]]
0.8
             precision    recall  f1-score   support

      crude       0.78      0.98      0.87        63
   money-fx       0.82      0.77      0.80        86
      trade       0.79      0.70      0.74        86

avg / total       0.80      0.80      0.80       235

